<a href="https://colab.research.google.com/github/swapnilmn/Assignment_1-CS6910/blob/master/Question_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible such that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

# Libraries 

In [4]:
from keras.datasets import fashion_mnist, mnist
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from sklearn.model_selection import train_test_split 
import pickle
import pandas as pd
import os
from tqdm import tqdm_notebook 
from sklearn.preprocessing import OneHotEncoder

In [6]:
def preprocess(data):

    if data == 'fashion_mnist':
        (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    elif data == 'mnist':
        (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    else:
        raise ValueError('Invalid dataset name')

    X_train, X_val, Y_train, Y_val = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)
    X_train= X_train / 255.0
    X_val =X_val / 255.0
    X_test = test_images / 255.0
    X_train= X_train.reshape(len(X_train), -1)
    X_val =X_val.reshape(len(X_val), -1)
    X_test=X_test.reshape(len(X_test), -1)
    enc = OneHotEncoder()
    Y_train =enc.fit_transform(np.expand_dims(Y_train, 1)).toarray()
    Y_val =enc.transform(np.expand_dims(Y_val, 1)).toarray()
    Y_test= enc.transform(np.expand_dims(test_labels, 1)).toarray()
    return X_train, X_val, X_test, Y_train, Y_val, Y_test
X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess('fashion_mnist')

#X_train, X_val, X_test, Y_train, Y_val, Y_test = X_train[0:100, :], X_val[0:10, :], X_test[0:30, :], Y_train[0:100, :], Y_val[0:10, :], Y_test[0:30, :]
print(Y_train.shape, Y_val.shape, Y_test.shape)
print(X_train.shape, X_val.shape, X_test.shape)

(54000, 10) (6000, 10) (10000, 10)
(54000, 784) (6000, 784) (10000, 784)
